In [38]:
# the following code allows you to import function from
# jupyter notebook in other directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [39]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb

IndexError: list index out of range

In [40]:
class ChiangOGWInvariant():
    def __init__(self, use_recursion_a=True, ogw_2_0_g3=1, ogw_1_1=3, debug=False):
        self.use_recursion_a = use_recursion_a
        self.ogw_1_0_g2 = 1/4
        self.ogw_2_0_g3 = ogw_2_0_g3
        self.ogw_1_1 = ogw_1_1
        self.debug = debug

    @functools.lru_cache(maxsize=None)
    def chiang_ogw_invariant(self, n, b, k, A, layer=0):
        #print(f"b:{b}, k:{k}, A:{A}")
        if isinstance(A, list):
            A = ClassList.compress(A, n)

        l = len(A)      
        if A.invalid_class_list:
            return 0    

        # wall-crossing theorem (Elad)
        if l > 0 and A[0] == -1:
            return -self.chiang_ogw_invariant(n, b, k+1, A[:-1]) 

        #degree axiom (Elad)
        if 2*sum(A) != (n - 3 + 2*b + k + 2*l - k*n):  
            # The maslov index is twice the chern class n+1 for holomorphic spheres. 
            # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
            # p 595 mcduff 
            #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
            if layer == 1:
                print("degree axiom fails")
            return 0    

        # other
        #if k == 0 and is_even(b):
        #    return 0    

        # zero axiom
        if b == 0:
            if k == 1 and l == 1 and A[0] == 0:
                return -1
            else:
                return 0    

        # unit axiom (May)
        # handle all beta = 0 cases
        if l > 0 and A[0] == 0:
            if b == 0 and k == 1 and l == 1:
                return -1
            else:
                return 0    

        # divisor axiom 
        if l > 0 and A[0] == 1:
            #A.pop(0)
            return (b/4)*self.chiang_ogw_invariant(n, b, k, A[1:])   

        # base cases
        if b == 1 and k == 1 and A == []:
            return self.ogw_1_1

        if b == 1 and k == 0 and A == [2]:
            return self.ogw_1_0_g2
        """

        if b == 2 and k == 2 and A == []:
            a = self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))^4 - 1
            b = self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([1, 3], n)) -\
                self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))*\
                    self.chiang_ogw_invariant(n, 1, 0, ClassList.compress([1, 2], n))
            return a/b
        """
        

        """
        if b == 2 and k == 0 and A == [2, 2]:
            return (-1 + self.chiang_ogw_invariant(n, 2, 1, ClassList.compress([2], n))^2)\
                    /(self.chiang_ogw_invariant(n, 2, 2, ClassList.compress([], n)))
        """
        

        """
        if b == 2 and k == 1 and A == [2]:
            return -self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))\
                    *self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))
        """
        
        if b == 2 and k == 0 and A == [3]:
            return self.ogw_2_0_g3
             

        # recursion (a)
        if l >= 2 and not (k >= 1 and not self.use_recursion_a):
            cor = self.chiang_ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
            col = 0
            ool = 0
            oor = 0
            A_0 = A[0]
            A_1 = A[1]  

            for t in A[2:].bipart():
                I1 = t[0]
                I2 = t[1]
                bipart_coeff = ClassList.bipart_coeff(A[2:], I1)    

                for i in range(0, n+1):
            
                    L1_cor = [1, A_1, i] + I1
                    L2_cor = [n-i, A_0-1] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        cor += bipart_coeff \
                            * (-1)^(b_*(n+1)/2) *  gw_invariant(n, b_, L1_cor) \
                            *  self.chiang_ogw_invariant(n, b1, k, L2_cor)   

                for i in range(0, n+1):
                
                    L1_col = [1, A[0] - 1, i] + I1 
                    L2_col = [n-i, A[1]] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        col += bipart_coeff \
                            * (-1)^((n+1)*b_/2)* gw_invariant(n, b_, L1_col) \
                            * self.chiang_ogw_invariant(n, b1, k, L2_col)    

                L1_ool = [1, A[0]-1] + I1
                L2_ool = [A[1]] + I2
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1
                        ool += bipart_coeff \
                            * binomial(k, k1)*self.chiang_ogw_invariant(n, b1, k1, L1_ool) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_ool)   

                L1_oor = [1, A[1]] + I1
                L2_oor = [A[0] - 1] + I2    
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1 

                        oor += bipart_coeff \
                            * binomial(k, k1) * self.chiang_ogw_invariant(n, b1, k1, L1_oor) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_oor)   

            result = cor - col - oor + ool
            if self.debug:
                print(f"recursion a: {result}")
            return result
        
        # recursion (c)
        if l >= 1 and k >= 1:
            #print(f"used c, A:{A}")
            #print(f"(c): b:{b}, k:{k}")
            co = 0
            ool = 0
            oor = 0
            A_0 = A[0]  

            for t in A[1:].bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A[1:], I1)    

                for i in range(0, n+1):
                    for b1 in range(0, b): # make sure that b1 is never b
                        b_ = 0 # \hat{\beta}
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue

                        co += bipart_coeff \
                            * gw_invariant(n, b_, [A_0 - 1, 1, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k, [n - i] + I2)   

                for b1 in range(1, b+1): # zero when b1 = 0
                    b2 = b - b1 

                    for k1 in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        ool += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1, [A_0 - 1, 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 2, I2)   

                for b1 in range(1, b+1): # exclude b1 = 0, which is zero, but would still cause infinite recursion
                    b2 = b - b1

                    for k1 in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        oor += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1 + 1, [A_0 - 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, [1] + I2) 
                        
            #print(f"recursion c,  b:{b}, k:{k}, A:{A}, co:{co}, ool:{ool}, oor:{oor}")

            result =  -co + ool - oor
            if self.debug:
                print(f"recursion c : {result}")
            return result   

        
        # recursion (d)
        if k >= 2:
            co = 0
            ool = 0
            oor = 0
            
            for t in A.bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A, I1)    

                for i in range(0, n+1):
                    for b1 in range(0, (b+2)+1): # make sure that b1 is never b
                        b_ = 0
                        if ((b + 2) - b1) % 4 == 0:
                            b_ = ((b + 2) - b1) / 4
                            if b_ == 0:
                                continue
                        else:
                            continue
                        co += bipart_coeff \
                            * gw_invariant(n, b_, [2, 2, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k-1, [n - i] + I2)
                        
                for b1 in range(2, b+1):
                    b2 = (b + 2) - b1
                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        
                        if b1 == b and k1 == k - 2:
                            continue
                            
                        ool += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+2, I1) \
                            * self.chiang_ogw_invariant(n, b2, k2, [2, 2] + I2)
                
                for b1 in range(2, b+1): # zero when b1 = 0
                    b2 = (b + 2) - b1 

                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        oor += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+1, [2] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2+1, [2] + I2)
                    
            
            result = (co - ool + oor)/(self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([2, 2], n)))       
            if self.debug:
                print(f"recursion d: {result}")
            return result   

        print(f"unexpected situation is , b: {b}, k: {k}, A:{A.c_list}, l: {l}")
        return 0